<a href="https://colab.research.google.com/github/ttenett/wordchain/blob/master/%EC%A6%9D%EC%83%81_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 30.3 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame
import numpy as np

In [9]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스_DS_DE_33/0_2ndpj/병명_증상 정보 data - Total.csv')

In [ ]:
df.head()

,담당자,부위,증상,질환명,진료과,TAG
0,박장훈,복부,신생아의 98%는 출생 후 24시간 내에 짙은 녹색의 태변을 배설합니다. 아기의 항...,선천성 거대결장증,소아외과,"히르쉬스프룽병 ,선청성대장질환 ,거대결장 ,대장조영술검사"
1,박장훈,복부,간에서 만들어진 담즙은 담도를 통해 소장으로 분비돼 소화를 돕는 데 사용됩니다. 그...,담도폐쇄증,"소화기내과, 간담췌외과, 소아외과","담도, 담도폐쇄증 ,billiary atresia ,간경변증 ,간이식 ,담도조영술 ..."
2,박장훈,복부,"조기 간암의 증상으로 피로감, 전신 쇠약감, 소화 불량, 상복부에 느껴지는 뻐근함,...",간암,"간담췌외과, 방사선종양학과, 혈액종양내과, 소화기내과, 간암센터","피로감, 구토 ,복통 ,황달 ,메스꺼움 ,식욕부진 ,체중감소 ,AFP ..."
3,박장훈,복부,"대표적인 증상으로는 복통, 설사, 체중 감소를 들 수 있습니다. 특히 10~20대 ...",크론병 Crohn disease,"소화기내과, 소아소화기영양과","복통 ,설사 ,식욕부진 ,체중감소 ,치루 ,장폐색 국소성 장염 ,궤양성 ..."
4,박장훈,복부,비만은 체내 근육량에 비해 지방조직이 과다한 상태를 의미합니다. 대한비만학회에서는 ...,고도비만 Severe obesity,"내분비내과, 가정의학과, 내분비당뇨병센터, 비만대사수술센터","빈혈, 속 쓰림 ,오심, 구토 ,복통 ,체중감소 ,위암개복수술 ,위암합병증 ..."


In [10]:
df.isnull()

,담당자,부위,증상,질환명,진료과,TAG
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
328,False,False,False,False,False,False
329,False,False,False,False,False,False
330,False,False,False,False,False,False
331,False,False,False,False,False,False


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   담당자     333 non-null    object
 1   부위      333 non-null    object
 2   증상      333 non-null    object
 3   질환명     333 non-null    object
 4   진료과     333 non-null    object
 5   TAG     333 non-null    object
dtypes: object(6)
memory usage: 15.7+ KB


In [12]:
# 텍스트 정제
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

In [13]:
# 문장 토큰화
def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
    return nouns

def tokenize(df):
    tokenizer = Mecab(dicpath='/usr/local/lib/mecab/dic/mecab-ko-dic')
    processed_data = []
    for sent in tqdm(df['text']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

In [7]:
# 토큰화
def save_processed_data(processed_data):
    with open("tokenized_data_"+df, 'w', newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in processed_data:
            writer.writerow(data)

# 형태소 분석 및 명사 추출
docs_n=[]
for doc in tqdm(df['증상']):
    doc_n = list(term for term in komoran.nouns(doc) if len(term)>0) #형태소 분석_1글자 이상의 명사 추출  #for문 내 if문 구조
    docs_n.append(doc_n) #이모티콘 등의 특수문자를 처리하는 경우, try except UnicodeDecodeError문 추가
#docs_n   #출력

NameError: name 'df' is not defined